In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

> # **Import packages and libraries**

In [ ]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import nltk
import pickle

from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

> # **EDA**

### Đọc file dữ liệu

In [ ]:
train = pd.read_csv('../input/quora-insincere-questions-classification/train.csv')
test = pd.read_csv('../input/quora-insincere-questions-classification/test.csv')

Đầu tiên, hãy nhìn qua về tập dữ liệu train trước

In [ ]:
train.head()

In [ ]:
train.info()

Bộ dữ liệu có **3 cột** và **1306122 non-null bản ghi (hàng)**   
Cột thứ 1 là '**qid**': id của từng câu hỏi  
Cột thứ 2 là '**question_text**': chứa tất cả các câu hỏi   
Cột thứ 3 là '**target**': nhãn của từng câu hỏi

Đếm số nhãn và số lượng của từng nhãn

In [ ]:
labels = Counter(train['target']).keys()
amounts = Counter(train['target']).values()
labels, amounts

#### Có 2 nhãn là: [0, 1]  với số lượng lần lượt là:   
| **Nhãn '0'** | **Nhãn '1'** |  
| :-------: | --------: |  
| **1225312**   | **80810**     |

### Visualize data

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
_labels = ['0', '1']
colors_list = ['#5cb85c','#5bc0de']
ax.bar(_labels, amounts, color=colors_list)

plt.xlabel('Label')
plt.ylabel('Amount')

for i in ax.patches:
    height = i.get_height()
    width = i.get_width()
    x, y = i.get_xy()
    ax.annotate(f'{height/1306122:.0%}', (x + width/2, height), ha='center')

plt.show()

In [ ]:
sincere_percent= (len(train.question_text[train['target'] == 0]) /  len(train['question_text']) * 100)
insincere_percent= (len(train.question_text[train['target'] == 1]) / len(train['question_text']) * 100)
sincere_percent, insincere_percent

In [ ]:
__labels = 'Sincere', 'Insincere'
sizes = [sincere_percent, insincere_percent]
explode = (0.1, 0)  # explode 1st slice

plt.pie(sizes, explode=explode, labels=__labels, colors=colors_list,
autopct='%1.1f%%', shadow=True, startangle=140)

plt.axis('equal')
plt.show()

Có thể thấy là dữ liệu train có nhãn rất lệch:  
Nhãn dương '1' chỉ chiếm 6,2% tổng số nhãn.  
Điều này sẽ là một vấn đề lớn cần được giải quyết để mô hình có thể trở thành 1 classifier tốt.

### Analyzing text statistics

Biểu đồ số ký tự có trong từng câu

In [ ]:
train['question_text'].str.len().hist()

Có thể thấy là các câu có độ dài thường từ 1->100 kí tự

Histogram về Số lượng từ trong 1 câu 

In [ ]:
train['question_text'].str.split().map(lambda x: len(x)).hist()

Và 1 câu thường có 1->12 từ, ngoài ra, các câu có 13->30 từ cũng có số lượng tương đối

Tạo ra 1 corpus  
Corpus: 1 list chứa tất cả các từ có trong tất cả các câu hỏi ở train['question_text']

In [ ]:
corpus = []
question = train['question_text'].str.split()
question = question.values.tolist()
corpus = [word for q in question for word in q]

In [ ]:
len(corpus), corpus[:20]

Stopword thường là các từ được sử dụng phổ biến trong văn bản cũng như là trong giao tiếp hàng ngày. Các từ stopword tiếng anh thường là 'a', 'an', 'the',... Tuy nhiên, các từ này thường không mang nhiều ý nghĩa trong việc xác định đặc tính của câu, mà chúng thường được sử dụng rất nhiều cho nên sẽ gây ảnh hưởng tới bước feature-extraction.  
Ta sẽ in ra các stopword được sử dụng ở trong tập dữ liệu train để đánh giá về số lượng của chúng

Trước tiên, vì các câu hỏi đều là câu hỏi bằng tiếng anh nên tập stopwords em sử dụng sẽ là list stopwords với ngôn ngữ là 'english' từ thư viện nltk (thư viện Natural Language Tool Kit - 1 thư viện Python phổ biến được sử dụng trong các bài toán NLP) 

In [ ]:
stop_words = stopwords.words('english')

In [ ]:
print(len(stop_words), stop_words)

In [ ]:
# stopword_dict -> sw_dict
from collections import defaultdict
sw_dict = defaultdict(int)
for word in corpus:
    if word in stop_words:
        sw_dict[word]+=1

In [ ]:
print(len(sw_dict), sw_dict)
print('The amount of stopwords: ', sum(sw_dict.values()))
print('Total words in corpus:', len(corpus))

Có 167 trên 179 stopword được dùng và số lượng stopword này là vô cùng nhiều: Chiếm 38% số lượng từ.  
Vì vậy, trong phần Tiền xử lý tiếp theo, chúng ta sẽ loại bỏ đi tất cả các stopword này.

Một vài biểu đồ trực quan hóa

In [ ]:
def plot_top_stopwords_barchart(texts):
    ques = texts.str.split()
    ques = ques.values.tolist()
    corpus = [word for q in ques for word in q]
    from collections import defaultdict
    dic = defaultdict(int)
    for word in corpus:
        if word in stop_words:
            dic[word]+=1
            
    top = sorted(dic.items(), key=lambda x:x[1],reverse=True)[:10] 
    x,y = zip(*top)
    plt.bar(x,y)
    plt.title('Top 10 stopword xuất hiện nhiều nhất')

In [ ]:
plot_top_stopwords_barchart(train['question_text'])

Vì đặc trưng của 1 câu trong văn bản có thể không chỉ được thể hiện qua 1 từ mà là còn từ cụm 2, 3 từ cho nên chúng ta sẽ:  
Visualize các từ đơn, từ đôi, cụm 3 từ xuất hiện nhiều trong dữ liệu:

In [ ]:
# Sử dụng hàm Counter từ thư viện collections để thống kê số lượng các từ đơn giản hơn
counter = Counter(corpus)
most = counter.most_common()

x,y= [],[]
for word,count in most[:40]:
    if (word not in stop_words):
        x.append(word)
        y.append(count)
        
sns.barplot(x=y,y=x).set(title='Các từ khác (không phải stopword) xuất hiện nhiều trong dữ liệu')

In [ ]:
# Hàm tìm các từ đôi trong từng câu hỏi từ bộ dữ liệu
def get_top_ngram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(n, n)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) 
                  for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:10]

In [ ]:
# Top từ đôi xuất hiện nhiều trong tập dữ liệu câu hỏi
top_bi_grams = get_top_ngram(train['question_text'],n=2)
x,y = map(list,zip(*top_bi_grams)) 
sns.barplot(x=y,y=x).set(title='Các từ đôi xuất hiện nhiều nhất trong dữ liệu')

In [ ]:
# Top cụm 3 từ xuất hiện nhiều trong tập dữ liệu câu hỏi
top_tri_grams = get_top_ngram(train['question_text'],n=3)
x,y = map(list,zip(*top_tri_grams)) 
sns.barplot(x=y,y=x).set(title='Các cụm 3 từ xuất hiện nhiều nhất trong dữ liệu')

Có thế thấy, khi chưa tiền xử lý dữ liệu, các từ đơn, từ đôi, cụm 3 từ xuất hiện nhiều nhất thường là các từ mở đầu câu hỏi như 'What', 'How', 'what is', 'is the', 'what is the', 'what are the', ... Điều này là không khó hiểu vì bộ dữ liệu là về các câu hỏi thu thập trên Quora.  
Tuy nhiên, nếu để những từ này lại thì sẽ gây cản trở cho việc học của mô hình vì đây là những từ không mang nhiều ý nghĩa nhưng xuất hiện nhiều (giống như stopword).  
Vậy nên, em sẽ đi vào bước tiếp theo là bước Tiền xử lý

-------------------------------------------------------------------------------------------

Em sẽ thực hiện các bước **Tiền xử lý** cơ bản của bài toán NLP như:  
* Lower các từ
* Xóa các kí tự không phải là chữ cái
* Xóa bỏ stopword
* Lemmatation

> # **Tiền xử lý** 

In [ ]:
# Xóa bỏ các từ là stopword
def delStopwords(text):
    return ' '.join([w for w in word_tokenize(text) if not w in stop_words])

# Đưa các từ về dạng từ gốc của nó
wordnet = WordNetLemmatizer() 
def lemmatizeText(text):
    return ' '.join([wordnet.lemmatize(w, pos='v') for w in word_tokenize(text)])

In [ ]:
def clean_text(text):
    cleaned_text = re.sub('[\n]',' ',text)    #xóa các ký tự xuống dòng
    cleaned_text = re.sub('[A-Z]+', lambda m: m.group(0).lower(), cleaned_text)    #lower các ký tự 
    cleaned_text = re.sub('[^a-zA-Z]',' ',cleaned_text).strip()    #xóa các ký tự không phải là chữ cái
    cleaned_text = delStopwords(cleaned_text)    #xóa stopwords
    cleaned_text = lemmatizeText(cleaned_text)    #lemmatize các từ về danh noun
    return cleaned_text

Thực hiện tiền xử lý dữ liệu và chia cả dữ liệu thành tập câu hỏi và tập nhãn

In [ ]:
train_text = []
target = []
for i in range(len(train)):
    train_text.append(clean_text(train['question_text'][i]))
    target.append((train['target'][i]).astype('int32'))
    
test_text = []
for i in range(len(test)):
    test_text.append(clean_text(test['question_text'][i]))

In [ ]:
len(train_text), len(target), len(test_text)

In [ ]:
# with open('./train_text.pkl', 'wb') as train_text_file:
#     pickle.dump(train_text, train_text_file)
# with open('./target.pkl', 'wb') as target_file:
#     pickle.dump(target, target_file)
# with open('./test_text.pkl', 'wb') as test_text_file:
#     pickle.dump(test_text, test_text_file)

In [ ]:
# train_text = pd.read_pickle(r'../input/preprocessedtexts/texts.pkl')
# target = pd.read_pickle(r'../input/text-targets/targets.pkl')
# test_text = pd.read_pickle(r'../input/preprocessedtesttexts/test_texts.pkl')

Như vậy, ta đã có được tập dữ liệu sau khi **Tiền xử lý**  

In [ ]:
preprocessed_train = pd.DataFrame(train_text, columns=['preprocessed_train_text'])
preprocessed_test = pd.DataFrame(test_text, columns=['preprocessed_test_text'])

Xây dựng 1 corpus mới với dữ liệu đã Tiền xử lý

In [ ]:
new_corpus = []
quest = preprocessed_train['preprocessed_train_text'].str.split()
quest = quest.values.tolist()
new_corpus = [word for q in quest for word in q]

In [ ]:
counter = Counter(new_corpus)
most = counter.most_common()

x,y= [],[]
for word,count in most[:20]:
    x.append(word)
    y.append(count)

sns.barplot(x=y,y=x).set(title='Các từ xuất hiện nhiều trong dữ liệu đã Tiền xử lý')

In [ ]:
# Top từ đôi xuất hiện nhiều trong tập dữ liệu đã Tiền xử lý
top_bi_grams = get_top_ngram(preprocessed_train['preprocessed_train_text'],n=2)
x,y = map(list,zip(*top_bi_grams)) 
sns.barplot(x=y,y=x).set(title='Các từ đôi xuất hiện nhiều nhất trong dữ liệu đã tiền xử lý')

In [ ]:
# Top cụm 3 từ xuất hiện nhiều trong tập dữ liệu đã Tiền xử lý
top_tri_grams = get_top_ngram(preprocessed_train['preprocessed_train_text'],n=3)
x,y = map(list,zip(*top_tri_grams)) 
sns.barplot(x=y,y=x).set(title='Các cụm 3 từ xuất hiện nhiều nhất trong dữ liệu đã tiền xử lý')

Có thể thấy là sau khi tiền xử lý, các từ, cụm 2 từ xuất hiện nhiều không còn là các từ, cụm từ mang tính khởi đầu của một câu hỏi như 'What', 'I', 'How', 'Why', 'What is', 'is the','What is the', 'What are the' ... nữa.  
 
Điều này là rất quan trọng trong bước feature extraction.  

------------------------------------------------------------------------------------------------------------------------------------------------------------------

Tiếp theo sẽ là phần Feature Extraction  
...